# Imports

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# global imports (1 sec - 30 secs)
import random
import numpy as np
import torch
from accelerate import Accelerator
from sentence_transformers import SentenceTransformer
accelerator = Accelerator()
model = accelerator.prepare(SentenceTransformer("all-mpnet-base-v2", device="cuda")) #all-MiniLM-L6-v2
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
# Get the max token size
max_token_size = model.max_seq_length
print(f"Max token size: {max_token_size}")

Max token size: 384


In [9]:
# local imports (1 sec - 2 secs)
import loader
import distances
import algos
import data_processing as dp
import classifier

In [10]:
#clear all memory of the GPU
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.reset_peak_memory_stats(device)

# Database

The miscellaneous database consists of 20 copyright-free book from children's literature obtained from [The Project Gutenberg](https://www.gutenberg.org/ebooks/bookshelf/20).  
The Tom Swift database consists of 27 books of the series Tom Swift by Victor Appleton obtained from [The Project Gutenberg](https://www.gutenberg.org/ebooks/search/?query=victor+appleton&submit_search=Go%21).

In [11]:
#Choosing the database ("Miscellaneous", "Tom Swift") and the number of books to load (1 min)
nb_books = 25
mode = "chunks"
all_sentences = loader.load_books("Tom Swift", mode, max_token_size, tokenizer=model.tokenizer)
sentences = all_sentences[:nb_books]

In [12]:
#choose a random book and print 1 chunk
r = random.randrange(0, len(sentences)) #not included
r2 = random.randrange(0, len(sentences[r]))
for i in range(r2, r2+1):
    print(sentences[r][i])

Motor-Cycle," here was related how he came to possess that machine. A certain Mr. Wakefield Damon, an eccentric gentleman, who was always blessing himself, or something about him, owned the cycle, but he came to grief on it, and sold it to Tom very cheaply. Tom had a number of adventures on the wheel, and, after having used the motor to save a valuable patent model from a gang of unscrupulous men, the lad acquired possession of a power boat, in which he made several trips, and took part in many exciting happenings. Some time later, in company with John Sharp, an aeronaut, whom Tom had rescued from Lake Carlopa, after the airman had nearly lost his life in a burning balloon, the young inventor made a big airship, called the Red Cloud. With Mr. Damon, Tom made several trips in this craft, as set forth in the book, "Tom Swift and His Airship." It was after this that Tom and his father built a submarine boat, and went under the ocean for sunken treasure, and, following that trip Tom built 

# Embeddings
Here we use Sentence-BERT to embed the sentences in the database.

In [13]:
#embeddings of the sentences (1 min - 2 min)
sentence_embedding = [model.encode(sentences[r]) for r in range(len(sentences))] #cannot stack because different number of pages

# Minimizing the distances
Here we want to find an oprtimal order by maximizing the semantic proximity between neighboring sentences. We have $n!$ possible orderings, so we can't use brute force. Our problem is similar to the traveling salesman problem, which is NP-hard, so we can't solve it optimally, therefore we design some algorithms to try to find a local minimum instead.

In [14]:
#run the local minimum algorithms on a subset of the sentences of the book and compare the permutation distances (1 sec - 4 secs)
pairwise_dist = distances.pairwise_dist(sentence_embedding)
distances2 = pairwise_dist[0][:100,:100]

default_order, random_order = list(range(len(distances2))), np.random.permutation(len(distances2))

for algo in [(lambda x, y: y), algos.insertion_sort, algos.greedy_sort]:
    for order in [default_order, random_order]:
        for dist, dist_name in [(lambda o : distances.avg_consecutive_dist(o, distances2), "avg_consecutive_dist"), (lambda o : distances.avg_swap_dist(o, default_order), "avg_swap_dist"), (lambda o : distances.avg_R_dist(o, default_order), "avg_R_dist")]:
            ordered = algo(distances2, order)
            d = dist(order)
            print(f"{algo.__name__:15}{dist_name:25}{d}")
        print()

<lambda>       avg_consecutive_dist     0.3072151839733124
<lambda>       avg_swap_dist            0.0
<lambda>       avg_R_dist               0.0

<lambda>       avg_consecutive_dist     0.4523981511592865
<lambda>       avg_swap_dist            0.2351
<lambda>       avg_R_dist               1.0

insertion_sort avg_consecutive_dist     0.3072151839733124
insertion_sort avg_swap_dist            0.0
insertion_sort avg_R_dist               0.0

insertion_sort avg_consecutive_dist     0.4523981511592865
insertion_sort avg_swap_dist            0.2351
insertion_sort avg_R_dist               1.0

greedy_sort    avg_consecutive_dist     0.3072151839733124
greedy_sort    avg_swap_dist            0.0
greedy_sort    avg_R_dist               0.0

greedy_sort    avg_consecutive_dist     0.4523981511592865
greedy_sort    avg_swap_dist            0.2351
greedy_sort    avg_R_dist               1.0



The first algorithm improves significantly the swap distance compared to a random permutation.

# Classifier: 
### Heuristics
The two metrics discussed were the following: distance betweeen pages or probability that a page is before another
For the distance between pages, we don't have the ground truth so to train a model it's easier to start with a classifier that takes in two pages and outputs a probability that the first page is before the second.

![Classifier Architecture](../img/Classifier_architecture.png)

### Training and Testing Datasets

In [15]:
#split the sentences into discard, training, validation and testing sets keeping the order
print(sentence_embedding[0].shape)
sentences_train, sentences_val, sentences_test = dp.split_sentences(sentence_embedding, 1)
print(sentences_train[0].shape, sentences_val[0].shape, sentences_test[0].shape)

(152, 768)
torch.Size([121, 768]) torch.Size([16, 768]) torch.Size([15, 768])


In [16]:
#create the database of the pairs of a subset of sentences (30 sec - 1 min for 25 books and 100%)
X_train, y_train = dp.create_database(sentences_train)
X_val, y_val = dp.create_database(sentences_val)
X_test, y_test = dp.create_database(sentences_test)
print(X_train.shape, X_val.shape, X_test.shape)     #size n x (n-1)

c:\Users\victo\Documents\Scolaire\EPFL 2022-2024\Semester 3\Jawbone\src\data_processing.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentence_embeddings = torch.tensor(sentence_embeddings).to("cuda")


torch.Size([362852, 1536]) torch.Size([5732, 1536]) torch.Size([5316, 1536])


### PyTorch Classifier

In [17]:
#hyperparameters
input_dim = X_train[0].shape[0]
output_dim = 1
hidden_dim = 128
learning_rate_list = [0.1, 0.01, 0.001, 0.0001]
epochs_list = [10, 100, 1000]
L2_alphas = [0, 0.01, 0.001, 0.0001]

In [18]:
#Create the classifier
network = classifier.Classifier(input_dim, hidden_dim, output_dim, accelerator)

The best values for the hyperparameters were found to be:
- Number of epochs: 10 000
- Learning Rate: 0.001
- L2 Regularization: 0

In [19]:
#set the best hyperparameters
learning_rate = 0.001
epochs = 1000
L2_alpha = 0

In [20]:
#train the network with the best hyperparameters (1 mins - 1.5 min)
loss = network.train(X_train, y_train, X_val, y_val, epochs, learning_rate, L2_alpha, True)

Epoch 0: train loss 0.693450
Epoch 100: train loss 0.494742
Epoch 200: train loss 0.452546
Epoch 300: train loss 0.423044
Epoch 400: train loss 0.380826
Epoch 500: train loss 0.332385
Epoch 600: train loss 0.289746
Epoch 700: train loss 0.254822
Epoch 800: train loss 0.226111
Epoch 900: train loss 0.201926
Validation loss 0.486506


In [21]:
#test on the GPU
y_pred = network(X_test.float().cuda())
#BCE loss
loss = network.loss_fn()(y_pred, y_test.float().cuda())
#convert to numpy arrays and round predictions
y_test_array, y_pred_array = y_test.cpu().detach().numpy(), y_pred.cpu().detach().numpy().round()
#accuracy
accuracy = accuracy_score(y_test_array, y_pred_array)
#F1 score
F1 = f1_score(y_test_array, y_pred_array)
#AUC score
AUC = roc_auc_score(y_test_array, y_pred_array)

print("Number of values %d" % y_test_array.shape[0])
print("Test BCE loss %f" % loss.item())
print("Test accuracy %f" % accuracy)
print("Test F1 %f" % F1.item())
print("Test AUC %f" % AUC.item())

Number of values 5316
Test BCE loss 0.473978
Test accuracy 0.799661
Test F1 0.798028
Test AUC 0.799661


### Exploiting the classifier

In [34]:
#embed a new book (2.5 sec)
new_embedding = [model.encode(all_sentences[nb_books])]

In [35]:
#predict the pairwise page order in the new book using the network (2 secs)
reduced_embedding = dp.split_sentences(new_embedding, 1, 1)[0]
X2, y2 = dp.create_database(reduced_embedding)
y_pred2 = network(X2.float().cuda())

In [36]:
#BCE loss
loss = network.loss_fn()(y_pred2, y2.float().cuda())
#convert to numpy arrays and round predictions
y2_array, y_pred2_array = y2.cpu().detach().numpy(), y_pred2.cpu().detach().numpy().round()
#accuracy
accuracy = accuracy_score(y2_array, y_pred2_array)
#F1 score
F1 = f1_score(y2_array, y_pred2_array)
#AUC score
AUC = roc_auc_score(y2_array, y_pred2_array)

print("Number of values %d" % y2_array.shape[0])
print("Test BCE loss %f" % loss.item())
print("Test accuracy %f" % accuracy)
print("Test F1 %f" % F1.item())
print("Test AUC %f" % AUC.item())

Number of values 24180
Test BCE loss 0.629497
Test accuracy 0.761497
Test F1 0.763463
Test AUC 0.761497


In [30]:
#convert the predictions to a pairwise probability matrix
pairwise_probabilities = dp.pred_to_pairwise(y_pred2)
#compute the min weight transitivity closure of the pairwise order graph using Floyd-Warshall
#compute obtimal order the topological sort algorithm
pred_order = dp.weighted_transitivity_closure(pairwise_probabilities)

In [44]:
#compare with the actual order of masked pages
ground_truth_order = list(range(len(new_embedding[0])))
print("Swap distance: %f" % distances.avg_swap_dist(pred_order, ground_truth_order))
print("R distance: %f" % distances.avg_R_dist(pred_order, ground_truth_order))

Swap distance: 0.124877
R distance: 0.897436


# Transformer:
### Heuristics 
Full transformer network that takes in pages as tokens and outputs an order, and the loss function would be a distance between two permutations.

In [27]:
#end-to-end transformer model using swap distance or R-distance as a loss function

In [28]:
#ChatGPT approach:
"""
Ordering the pages of a book using a transformer model can be challenging, especially when dealing with limited token size. Here's how you can create a transformer model for this problem and address the token size limitation:

1. Data Preparation:

    Start by collecting a dataset of books with pages not in order. Each page should be a separate input example, and the pages should be represented as text.

2. Text Tokenization:

    Tokenize the text from each page into smaller units, such as words or subwords, using a tokenizer. Popular tokenization libraries like Hugging Face Transformers provide tokenizers that can handle large texts and split them into tokens without worrying about the token size limitation.

3. Sliding Window Approach:

    As you mentioned, most transformer models have a token size limitation. To overcome this limitation, you can use a sliding window approach. Split each page into overlapping segments or windows of tokens. This will allow you to work with manageable token sizes for your model.

4. Model Architecture:

    You can use a standard transformer architecture for this task. However, you might need to modify it slightly to account for the specific requirements of ordering pages. Your model should be capable of learning the relationships between pages and their optimal order.

5. Training:

    Train your transformer model on the dataset of disordered pages. You can use a contrastive loss function to ensure that the model learns to distinguish between correct and incorrect page orderings. This involves providing pairs of pages where one pair is in the correct order, and another pair is not.

6. Inference:

    When you want to order a book with disordered pages, you can feed the pages into your trained model. The model should provide a probability score or ranking for each possible page order. You can then select the order with the highest score as the predicted correct order.

7. Evaluation:

    To evaluate the model's performance, you can use metrics like mean squared error (MSE) or Kendall's Tau rank correlation to compare the predicted order with the ground truth order.

Keep in mind that this is a challenging NLP task, and it may require a significant amount of data and computational resources. Additionally, your sliding window approach should be carefully designed to minimize information loss while breaking down the text into manageable token-sized chunks.
"""

"\nOrdering the pages of a book using a transformer model can be challenging, especially when dealing with limited token size. Here's how you can create a transformer model for this problem and address the token size limitation:\n\n1. Data Preparation:\n\n    Start by collecting a dataset of books with pages not in order. Each page should be a separate input example, and the pages should be represented as text.\n\n2. Text Tokenization:\n\n    Tokenize the text from each page into smaller units, such as words or subwords, using a tokenizer. Popular tokenization libraries like Hugging Face Transformers provide tokenizers that can handle large texts and split them into tokens without worrying about the token size limitation.\n\n3. Sliding Window Approach:\n\n    As you mentioned, most transformer models have a token size limitation. To overcome this limitation, you can use a sliding window approach. Split each page into overlapping segments or windows of tokens. This will allow you to wor